# Autoencoder with multi-head attention using pytorch
Following the example from: <br>
https://medium.com/the-dl/transformers-from-scratch-in-pytorch-8777e346ca51

In [ ]:
import numpy as np
import torch

![attention](../assets/img/multihead_attention.png)